In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import os, sys, subprocess
import json
import pandas as pd
import datetime as dt
import pytz
import numpy as np
pd.set_option('display.max_columns', None)

##  What fields to get from county data
- Two Week Crude IR* (per 100k)
- Newly Confirmed Case Count
- Newly Reported Death Count

## Time frame for week

- Week start date: Saturday
- Week end date: Friday

In [110]:
def process_tables(file_path, neighborhood_column, colnames):
    """
    colnames = list of column names that you want to add
    neighborhood_column = the column name in the table that designates neighborhood
    
    """
    column_list = [neighborhood_column]
    column_list = column_list + colnames
    
    if neighborhood_column == "Community":
        df = pd.read_csv(file_path)
    else:
        df = pd.read_csv(file_path).iloc[:, 1:]
    
    df.dropna(axis = 0, subset= [neighborhood_column], inplace = True)

    # filter for los angeles neighborhoods
    la = df[(df[neighborhood_column].str.contains("Los Angeles")) | (df[neighborhood_column].str.contains("Unincorporated - Angeles National Forest"))]
    
    # Remove LA from neighborhood name
    la.loc[:, neighborhood_column] = la.loc[:, neighborhood_column].str.replace("Los Angeles - ", "")
    #la.loc[:, "geo_merge"] = la.loc[:, "geo_merge"].str.replace("Unincorporated - ", "")

    # Drop other county names
   # la = la[~la["geo_merge"].str.contains("Unincorporated")]
    la = la[~la[neighborhood_column].str.contains("Los Angeles County")]
    


    # filter data for the last week
    if "ep_date" in la:
        la["ep_date"] = pd.to_datetime(la["ep_date"])
        la = la.sort_values(by="ep_date")
        la_week = la.drop_duplicates(subset=[neighborhood_column], keep = "last")[column_list+ ["ep_date"] ]
        la_week=la_week.rename(columns={neighborhood_column:'Neighborhood',
                                    "ep_date": "Date"
                      })    
   
    elif "Date (Dose 1)" in la:
        la["Date (Dose 1)"] = pd.to_datetime(la["Date (Dose 1)"])
        la = la.sort_values(by="Date (Dose 1)")
        la_week = la.drop_duplicates(subset=[neighborhood_column], keep = "last")[column_list+ ["Date (Dose 1)"] ]
        la_week=la_week.rename(columns={neighborhood_column:'Neighborhood',
                                    "Date (Dose 1)": "Date"
                      })    
   
    else: 
        la_week = la[column_list]
        la_week=la_week.rename(columns={neighborhood_column:'Neighborhood'})    
   
        


    return la_week

In [ ]:
two_week = process_tables("data/LA_County_Covid19_CSA_14day_case_death_table.csv","geo_merge",  ["case_14day_rate", "adj_case_14day_rate"])
week = process_tables("data/LA_County_Covid19_CSA_7day_case_death_table.csv", "geo_merge", ["cases_7day", "death_7day"])
cumulative = process_tables("data/LA_County_Covid19_CSA_case_death_table.csv","geo_merge", ["cases_final", "deaths_final", "population"])
vax = process_tables("data/LAC_Vaccine_City_Data_old.csv", "Community", ["Cumulative 16+ (Dose 1)", "Cumulative Percentage 16+ (Dose 1)", "Cumulative Percentage 12-17 (Dose 1)", "Cumulative Percentage 65+ (Dose 1)"])

In [114]:
drop_names = ["East Los Angeles", "Lake Los Angeles"]

In [115]:
df = pd.merge(pd.merge(two_week, week, how="left", on = "Neighborhood"), cumulative, how="left",  on = "Neighborhood")
df = pd.merge(df, vax, how="left", on = "Neighborhood")
df = df[df["Neighborhood"] != "Angeles National Forest"]
df["Neighborhood"] = df["Neighborhood"].str.replace("Unincorporated - ", "")
df = df[~df["Neighborhood"].isin(drop_names)]
df = df.sort_values(by="Neighborhood")
df = df.rename(columns={"Date_x": "date_14day",
                        "Date_y": "date_7day"
    
})

# convert all to float 
df[["death_7day", "cases_final", "deaths_final", "population"]] = df[["death_7day", "cases_final", "deaths_final", "population"]].astype(float)


In [116]:
last_date = df["date_14day"].max()

In [117]:
prior_date = last_date + dt.timedelta(days = -6)

In [118]:
two_week_date = last_date + dt.timedelta(days = -14)

In [119]:
print(last_date)

2021-07-23 00:00:00


In [120]:
print(prior_date)

2021-07-17 00:00:00


In [121]:
# create trends
df["top_case_rate"]= df.loc[(df["cases_7day"] > 10) & (df["date_14day"] >= last_date)]["case_14day_rate"].rank(ascending=False)
df["top_new_cases"]= df["cases_7day"].rank(ascending=False)

In [122]:
# rename columns with dates
df = df.rename(columns={"cases_7day": f'New Cases {prior_date.strftime("%-m.%-d")} to {last_date.strftime("%-m.%-d")}',
                         "death_7day": f'New Deaths {prior_date.strftime("%-m.%-d")} to {last_date.strftime("%-m.%-d")}',
                         "case_14day_rate": f'Case Rate {two_week_date.strftime("%-m.%-d")} to {last_date.strftime("%-m.%-d")}',
                         "adj_case_14day_rate": f'Adjusted Case Rate {two_week_date.strftime("%-m.%-d")} to {last_date.strftime("%-m.%-d")}',
                        "cases_final": f'Cumulative Confirmed Case Count {last_date.strftime("%-m.%-d")}',
                         "deaths_final": f'Cumulative Death Count {last_date.strftime("%-m.%-d")}',
                        "Cumulative 16+ (Dose 1)": "persons_vaccinated", 
                        "Cumulative Percentage 16+ (Dose 1)": "percent_of_ppl_vaccinated",
                        "Cumulative Percentage 12-17 (Dose 1)": "percent_12_to_17_vaccinated",
                        "Cumulative Percentage 65+ (Dose 1)": "percent_65_plus_vaccinated"
                        

                        }
               )


In [123]:
df.shape

(139, 17)

In [126]:
df.to_csv("data/testing_neighborhoods.csv", index = False)

# Create areas

In [128]:
def process_area_case_rates(file_path, community, colnames):
    """
    colnames = list of column names that you want to add
    community = the column name in the table that designates neighborhood
    
    """
    column_list = [community]
    column_list = column_list + colnames
    
    df = pd.read_csv(file_path).iloc[:, 1:]
    df.dropna(axis = 0, subset= [community], inplace = True)
    last_week = df[df["ep_date"] == prior_date.strftime("%Y-%m-%d")]
    this_week = df[df["ep_date"] == last_date.strftime("%Y-%m-%d")]
    df = pd.concat([last_week, this_week])

    # filter for los angeles neighborhoods
    la = df[(df[community].str.contains("Los Angeles")) | (df[community].str.contains("Unincorporated - Angeles National Forest"))]
    
    # Remove LA from neighborhood name
    la.loc[:, community] = la.loc[:, community].str.replace("Los Angeles - ", "")

    # Drop other county names
    la = la[~la[community].str.contains("Los Angeles County")]
    


    two_weeks=la[column_list].rename(columns={community:'Neighborhood',
                                    "ep_date": "Date"
                      })    
    
    two_weeks = two_weeks.groupby("Neighborhood").sum().reset_index().rename(columns={'cases_7day':'two_week_cases',
                                    "death_7day": "two_week_deaths"
    
                                                                                     })
        


    return two_weeks

In [ ]:
two_week_cases = process_area_case_rates("data/LA_County_Covid19_CSA_7day_case_death_table.csv", "geo_merge", ["cases_7day", "death_7day"])

In [191]:
areas = pd.read_csv("data/area_mappings.csv")

In [192]:
df = pd.read_csv("data/testing_neighborhoods.csv")[["Neighborhood", "population"]]

In [193]:
df = pd.merge(df, pd.merge(two_week_cases, areas, on = "Neighborhood"))

In [194]:
area_df = df.groupby("Area").sum().reset_index()

In [196]:
area_df["percent_cases"]= area_df["two_week_cases"]/area_df["two_week_cases"].sum()
area_df["percent_pop"]= area_df["population"]/area_df["population"].sum()
area_df["case_rate"]= round((area_df["two_week_cases"]/area_df["population"])*100_000,0)

In [201]:
area_df.to_csv("data/area_summary.csv", index=False)

### Read in HV data

In [49]:
# read in hv data
hv = pd.read_csv("../Declan/Data/hv_data/hv_subset_2021.csv", low_memory = False)

In [50]:
# convert date to datetime object
#hv["result_datetime"] = pd.to_datetime(hv["result_datetime"])

hv["result_date"] = pd.to_datetime(hv["result_date"])

In [51]:
# grab the week start date (Saturday)
week_start_date = last_date - pd.DateOffset(days=6)

In [52]:
# filter data for the last week
hv_week = hv[(hv["result_date"]>= week_start_date.strftime("%Y-%-m-%-d"))&
    (hv["result_date"]<= last_date.strftime("%Y-%-m-%-d"))][["zip", "display_value"]]

In [53]:
# quick cleaning for zip column
hv_week["zip"] = hv_week["zip"].str.replace("`", "")
hv_week = hv_week[hv_week['zip'].astype(str).str.isdigit()]

In [54]:
# Group by zip code and get test result counts 
tests = pd.DataFrame(hv_week.groupby(["zip", "display_value"]).size()).reset_index().rename(columns={0:'count'})

In [55]:
# Pivot
tests = tests.pivot(index='display_value', columns='zip').reset_index().transpose()
tests = tests.reset_index().drop(columns='level_0')

In [56]:
# Get labels
labels = tests.iloc[0]

In [57]:
# drop first row
hv_table = tests[1:]
# rename column
hv_table.columns= labels

In [58]:
# reset index
hv_table = hv_table.set_index("")
# fill NA
hv_table = hv_table.fillna(0).astype("int")

In [59]:
# calculate total
hv_table['Total'] = hv_table['Negative']+hv_table['Positive']
# calculate test positivity
hv_table["Test Posivitiy"] = hv_table["Positive"]/hv_table["Total"]

In [60]:
hv_table = hv_table.reset_index().rename(columns = {hv_table.index.name: "Zip"})

In [61]:
hv_table["Date"] = pd.to_datetime(last_date, format="%d-%b-%Y")

In [62]:
hv_table.to_csv("data/test_positivity_zip.csv", index = False)